See 1. A Numerical Example in Valuing American Options by Simulation: A Simple Least-Squares Approach Francis A. Longstaff UCLA Eduardo S. Schwartz UCLA.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.linear_model import LinearRegression


def least_square_Monte_Carlo_test(paths,r,dt,T,payoff,K):
    value = payoff(paths[:,-1])
    display(pd.DataFrame(value))
    for col in paths.T[:-1][::-1]:# stepping backwards in time
      value *= np.exp(-r*dt) # discounting 
      ITMmask = col < K # selects paths ITM
      ITMpaths = col[ITMmask]
      ITMvalue = value[ITMmask]
      model = LinearRegression()
      X = np.column_stack((ITMpaths,ITMpaths**2))
      model.fit(X,ITMvalue) # LS of the continuation value
      interpolated = model.predict(X)
      exercised = payoff_put(ITMpaths,K)
      # value[mask] = np.maximum(interpolated,exercise) see note in paper
      value[ITMmask] = np.where(interpolated>exercised, value[ITMmask], exercised)

      print('model.intercept_',model.intercept_) 
      print('model.coef_',model.coef_)
      
      df = pd.DataFrame({'exercised': exercised,'interpolated': interpolated})
      display(df)

    return np.mean(value), np.std(value)/np.sqrt(len(value))
    
from parameters import parameters_stylized_longstaff, paths_longstaff

paths = paths_longstaff()
print("paths:")
display(pd.DataFrame(paths))

def payoff_put(S,K): return np.maximum(K-S,0)

par = parameters_stylized_longstaff()

val,std = least_square_Monte_Carlo_test(paths,par.r,par.dt,par.T,lambda S: payoff_put(S,par.K),par.K)
print(val,std)

paths:


,0,1,2,3
0,1.0,1.09,1.08,1.34
1,1.0,1.16,1.26,1.54
2,1.0,1.22,1.07,1.03
3,1.0,0.93,0.97,0.92
4,1.0,1.11,1.56,1.52
5,1.0,0.76,0.77,0.90
6,1.0,0.92,0.84,1.01
7,1.0,0.88,1.22,1.34


,0
0,0.00
1,0.00
2,0.07
3,0.18
4,0.00
5,0.20
6,0.09
7,0.00


model.intercept_ -1.0699876552910912
model.coef_ [ 2.98341063 -1.81357618]


,exercised,interpolated
0,0.02,0.036741
1,0.03,0.045898
2,0.13,0.117527
3,0.33,0.151969
4,0.26,0.156418


model.intercept_ 2.037512342379653
model.coef_ [-3.3354434   1.35645659]


,exercised,interpolated
0,0.01,0.013485
1,0.17,0.108749
2,0.34,0.286065
3,0.18,0.117009
4,0.22,0.152762


model.intercept_ 0.11443433004505696
model.coef_ [0. 0.]


,exercised,interpolated
0,0.1,0.114434
1,0.1,0.114434
2,0.1,0.114434
3,0.1,0.114434
4,0.1,0.114434
5,0.1,0.114434
6,0.1,0.114434
7,0.1,0.114434


0.11443433004505696 0.03922691623092562
